In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [2]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0)

search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)


class CalculatorInput(BaseModel):
    question: str = Field(description="The math question to answer.")


tools = [
    Tool.from_function(
        func=search.run,
        name = "Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
]

In [3]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.experimental.autoflow.schema import Schema, FlowSchemaOutputParser
from langchain import PromptTemplate
from langchain.experimental.autoflow.prompts import TASK_BREAKDOWN

schema = Schema.object(
    solvability_thought=Schema.string("The thought process you used to determine if the task is solvable"),
    solvable=Schema.boolean("Whether the task is solvable"),
    subtasks=Schema.array(Schema.string(), "The subtasks you identified; [] if task is not solvable"),
    reflection=Schema.string("Your reflection on the subtasks breakdown above, any advice to improve; if task is not solvable, or no improvement suggustion")
)

llm = ChatOpenAI(temperature=0, model="gpt-4")
output_parser = FlowSchemaOutputParser(flow_schema=schema)

prompt = PromptTemplate.from_template(
    TASK_BREAKDOWN,
    template_format="jinja2",
    output_parser=output_parser
)

llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True
)
result = llm_chain.predict_and_parse(
    task="What is the capital of the input country?",
    input_schema=Schema.object(
        country=Schema.string("The country to find the capital of")
    ),
    output_schema=Schema.object(
        capital=Schema.string("The capital of the input country")
    ),
    attempt_history=[],
    format_instructions=output_parser.get_format_instructions(),
    tools=tools
)
result



> Entering new LLMChain chain...
Prompt after formatting:
You are acted as a task breakdown expert. Your job is to breakdown a task into a list of subtasks that potentially can be solved by existing tools.
Subtasks form a dependency flow graph that solve the main task, so that one subtask may depends on the output of other subtasks.

[TOOL_DESCRIPTION]
You have access to a set of tools with their description:

- Search: useful for when you need to answer questions about current events

- Calculator: useful for when you need to answer questions about math


[TASK_DESCRIPTION]
The task you are working on is:
What is the capital of the input country?

[INPUT_CONTEXT]
Here is the input context given to you (necessary for solving the main task):
{
  "country": string, (The country to find the capital of)
}

[OUTPUT_SCHEMA]
Here is the output schema of the task:
{
  "capital": string, (The capital of the input country)
}

[PREVIOUS_ATTEMPTS]
Here are the previous attempts of breaking down 

{'solvability_thought': 'The main task requires finding the capital of a given country. The Search tool can be used to find information about current events, which includes finding the capital of a country.',
 'solvable': True,
 'subtasks': ['Use the Search tool to find the capital of the input country'],
 'reflection': 'The task can be solved using the Search tool, and the subtask list is straightforward. No improvements are needed.'}